# Libraries

In [1]:
import pandas as pd
import numpy as np
import math

import time

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from tqdm import tqdm

import os

# Retraction Watch

In [2]:
rw = pd.read_excel('./retractions_data/retraction_watch_database.xlsx')
rw.head()

,Record ID,Title,Subject,Institution,Journal,Publisher,Country,Author,URLS,ArticleType,RetractionDate,RetractionDOI,RetractionPubMedID,OriginalPaperDate,OriginalPaperDOI,OriginalPaperPubMedID,RetractionNature,Reason,Paywalled,Notes
0,47271,Binding of DCC by Netrin-1 to Mediate Axon Gui...,(BLS) Biology - Cellular;(BLS) Biology - Gener...,Departments of Anatomy and of Biochemistry and...,Science,American Association for the Advancement of Sc...,United States,Elke Stein;Yimin Zou;Mu-ming Poo;Marc Tessier-...,https://retractionwatch.com/2023/08/31/stanfor...,Research Article;,2023-08-31 00:00:00,10.1126/science.adk1521,0.0,2001-03-09 00:00:00,10.1126/science.1059391,11239160.0,Retraction,+Investigation by Company/Institution;+Manipul...,No,NaN
1,47270,Hierarchical Organization of Guidance Receptor...,(BLS) Biochemistry;(BLS) Biology - General;(BL...,Department of Anatomy and Department of Bioche...,Science,American Association for the Advancement of Sc...,United States,Elke Stein;Marc Tessier-Lavigne,https://retractionwatch.com/2023/08/31/stanfor...,Research Article;,2023-08-31 00:00:00,10.1126/science.adk1517,0.0,2001-02-08 00:00:00,10.1126/science.1058445,11239147.0,Retraction,+Duplication of Image;+Investigation by Compan...,No,NaN
2,47243,Therapeutic potential of targeting IRES-depend...,(BLS) Biochemistry;(BLS) Biology - Cancer;(BLS...,"Division of Hematology-Oncology, UCLA-Greater ...",Oncogene,Springer - Nature Publishing Group,United States,Y Shi;Y Yang;C Bardeleben;B Holmes;J Gera;Alan...,NaN,Research Article;,2023-08-31 00:00:00,10.1038/s41388-023-02820-5,0.0,2015-05-11 00:00:00,10.1038/onc.2015.156,25961916.0,Retraction,+Concerns/Issues About Data;+Concerns/Issues A...,No,see also: https://pubpeer.com/publications/704...
3,47233,A classifier based on 273 urinary peptides pre...,(BLS) Biochemistry;(HSC) Medicine - Cardiovasc...,"Department of Nephrology, The Third Affiliated...",Journal of Hypertension,Wolters Kluwer - Lippincott Williams & Wilkins,China,Lirong Lin;Chunxuan Wang;Jiangwen Ren;Mei Mei;...,NaN,Research Article;,2023-08-30 00:00:00,10.1097/HJH.0000000000003551,37642599.0,2023-08-01 00:00:00,10.1097/HJH.0000000000003467,37199562.0,Retraction,+Concerns/Issues About Results;+Investigation ...,No,see also https://journals.lww.com/jhypertensio...
4,47227,"Age, Gender Demographics and Comorbidity Preva...",(HSC) Biostatistics/Epidemiology;(HSC) Medicin...,"Department of Orthopaedics, Dhanalakshmi Srini...",Journal of Coastal Life Medicine,Journal of Coastal Life Medicine,India,S Venkatesh Kumar;Mohith Singh;Gowtham Singh;K...,NaN,Research Article;,2023-08-30 00:00:00,unavailable,0.0,2023-01-01 00:00:00,unavailable,0.0,Retraction,+Notice - Lack of;+Withdrawal;,No,"date of retraction unknown, article title repl..."


In [3]:
rw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42700 entries, 0 to 42699
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Record ID              42700 non-null  int64  
 1   Title                  42700 non-null  object 
 2   Subject                42700 non-null  object 
 3   Institution            42699 non-null  object 
 4   Journal                42700 non-null  object 
 5   Publisher              42700 non-null  object 
 6   Country                42700 non-null  object 
 7   Author                 42700 non-null  object 
 8   URLS                   21687 non-null  object 
 9   ArticleType            42700 non-null  object 
 10  RetractionDate         42700 non-null  object 
 11  RetractionDOI          42209 non-null  object 
 12  RetractionPubMedID     37599 non-null  float64
 13  OriginalPaperDate      42700 non-null  object 
 14  OriginalPaperDOI       40173 non-null  object 
 15  Or

In [4]:
rw['RetractionDate'] = pd.to_datetime(rw['RetractionDate'], errors='coerce') #, infer_datetime_format=True
rw['OriginalPaperDate'] = pd.to_datetime(rw['OriginalPaperDate'])

# Journals to consider

In [5]:
journals = pd.read_csv('../scimagojr_2022.csv', sep=';')
journals

,Rank,Sourceid,Title,Type,Issn,SJR,SJR Best Quartile,H index,Total Docs. (2022),Total Docs. (3years),...,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Region,Publisher,Coverage,Categories,Areas
0,1,28773,Ca-A Cancer Journal for Clinicians,journal,"15424863, 00079235","86,091",Q1,198,44,118,...,30318,85,"299,99","97,00",United States,Northern America,Wiley-Blackwell,1950-2022,Hematology (Q1); Oncology (Q1),Medicine
1,2,29431,Quarterly Journal of Economics,journal,"00335533, 15314650","36,730",Q1,292,36,122,...,2141,122,"14,83","66,61",United Kingdom,Western Europe,Oxford University Press,1886-2022,Economics and Econometrics (Q1),"Economics, Econometrics and Finance"
2,3,20315,Nature Reviews Molecular Cell Biology,journal,"14710072, 14710080","34,201",Q1,485,121,328,...,13331,156,"35,47","89,29",United Kingdom,Western Europe,Nature Publishing Group,2000-2022,Cell Biology (Q1); Molecular Biology (Q1),"Biochemistry, Genetics and Molecular Biology"
3,4,18434,Cell,journal,"00928674, 10974172","26,494",Q1,856,420,1637,...,67791,1440,"43,80","65,74",United States,Northern America,Cell Press,1974-2022,"Biochemistry, Genetics and Molecular Biology (...","Biochemistry, Genetics and Molecular Biology"
4,5,15847,New England Journal of Medicine,journal,"00284793, 15334406","26,015",Q1,1130,1410,4561,...,133956,1854,"33,93","10,21",United States,Northern America,Massachussetts Medical Society,1945-2022,Medicine (miscellaneous) (Q1),Medicine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18031,18032,17500154901,Progress in Molecular Biology and Translationa...,journal,"18771173, 18780814",NaN,-,110,90,314,...,1170,0,"2,72","108,88",Netherlands,Western Europe,Academic Press Inc.,2008-2022,Molecular Biology; Molecular Medicine,"Biochemistry, Genetics and Molecular Biology"
18032,18033,25192,Reviews of Environmental Contamination and Tox...,journal,01795953,NaN,-,94,22,99,...,441,0,"4,40","151,68",United States,Northern America,Springer New York,1987-2022,"Health, Toxicology and Mutagenesis; Medicine (...",Environmental Science; Medicine
18033,18034,5700155185,Voprosy Istorii (discontinued),journal,00428779,NaN,-,5,168,1316,...,38,1316,"0,03","7,96",Russian Federation,Eastern Europe,"Rossiiskaya Akademiya Nauk, Institut Istorii (...","1965, 1972, 1975, 1978-1982, 1985, 1988, 1999-...",History; Medicine (miscellaneous),Arts and Humanities; Medicine
18034,18035,21100873483,Wisdom (discontinued),journal,18293824,NaN,-,7,66,180,...,76,180,"0,45","25,61",Armenia,Eastern Europe,Khachatur Abovyan Armenian State Pedagogical U...,2018-2022,Philosophy,Arts and Humanities


In [6]:
# Calculate the threshold for the top 10%
threshold = int(0.10 * len(journals))

# Select the top 10% of rows
top_10_percent = journals.head(threshold)
top_10_percent

,Rank,Sourceid,Title,Type,Issn,SJR,SJR Best Quartile,H index,Total Docs. (2022),Total Docs. (3years),...,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Region,Publisher,Coverage,Categories,Areas
0,1,28773,Ca-A Cancer Journal for Clinicians,journal,"15424863, 00079235","86,091",Q1,198,44,118,...,30318,85,"299,99","97,00",United States,Northern America,Wiley-Blackwell,1950-2022,Hematology (Q1); Oncology (Q1),Medicine
1,2,29431,Quarterly Journal of Economics,journal,"00335533, 15314650","36,730",Q1,292,36,122,...,2141,122,"14,83","66,61",United Kingdom,Western Europe,Oxford University Press,1886-2022,Economics and Econometrics (Q1),"Economics, Econometrics and Finance"
2,3,20315,Nature Reviews Molecular Cell Biology,journal,"14710072, 14710080","34,201",Q1,485,121,328,...,13331,156,"35,47","89,29",United Kingdom,Western Europe,Nature Publishing Group,2000-2022,Cell Biology (Q1); Molecular Biology (Q1),"Biochemistry, Genetics and Molecular Biology"
3,4,18434,Cell,journal,"00928674, 10974172","26,494",Q1,856,420,1637,...,67791,1440,"43,80","65,74",United States,Northern America,Cell Press,1974-2022,"Biochemistry, Genetics and Molecular Biology (...","Biochemistry, Genetics and Molecular Biology"
4,5,15847,New England Journal of Medicine,journal,"00284793, 15334406","26,015",Q1,1130,1410,4561,...,133956,1854,"33,93","10,21",United States,Northern America,Massachussetts Medical Society,1945-2022,Medicine (miscellaneous) (Q1),Medicine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1798,1799,19900192175,Journal of Topology,journal,"17538416, 17538424","1,575",Q1,28,56,119,...,137,119,"1,11","36,68",United Kingdom,Western Europe,John Wiley and Sons Ltd,2010-2022,Geometry and Topology (Q1),Mathematics
1799,1800,21100875643,Materials Chemistry Frontiers,journal,20521537,"1,575",Q1,70,367,1245,...,8782,1233,"7,21","63,92",United Kingdom,Western Europe,Royal Society of Chemistry,2017-2022,Materials Chemistry (Q1); Materials Science (m...,Materials Science
1800,1801,15061,Agricultural Systems,journal,"0308521X, 18732267","1,574",Q1,126,166,607,...,4408,601,"7,02","72,49",United Kingdom,Western Europe,Elsevier BV,1976-2022,Agronomy and Crop Science (Q1); Animal Science...,Agricultural and Biological Sciences
1801,1802,26112,Frontiers of Hormone Research,journal,03013073,"1,574",Q1,42,0,45,...,156,41,"0,92","0,00",Switzerland,Western Europe,S. Karger AG,"1975, 1977, 1984, 1996-1997, 1999-2002, 2004-2...","Endocrinology (Q1); Endocrinology, Diabetes an...","Biochemistry, Genetics and Molecular Biology; ..."


In [7]:
top_10_percent['Issn']

0       15424863, 00079235
1       00335533, 15314650
2       14710072, 14710080
3       00928674, 10974172
4       00284793, 15334406
               ...        
1798    17538416, 17538424
1799              20521537
1800    0308521X, 18732267
1801              03013073
1802    14617307, 13505076
Name: Issn, Length: 1803, dtype: object

In [8]:
top_10_percent[top_10_percent['Issn'].str.contains(',')]['Issn']

0       15424863, 00079235
1       00335533, 15314650
2       14710072, 14710080
3       00928674, 10974172
4       00284793, 15334406
               ...        
1795    15565653, 00150282
1796    19360533, 19360541
1798    17538416, 17538424
1800    0308521X, 18732267
1802    14617307, 13505076
Name: Issn, Length: 1191, dtype: object

In [9]:
top_10_percent[top_10_percent['Issn'].str.count(',') > 1]['Issn']

143     15383598, 00987484, 00029955
263     03785912, 01662236, 1878108X
304     13624326, 03765067, 09680004
1545    08203946, 00084409, 14882329
Name: Issn, dtype: object

In [10]:
journals = top_10_percent['Issn'][0:100].values
journals

array(['15424863, 00079235', '00335533, 15314650', '14710072, 14710080',
       '00928674, 10974172', '00284793, 15334406', '1546170X, 10788956',
       '10575987, 15458601', '15461696, 10870156', '20588437', '00028282',
       '1474175X, 14741768', '14764687, 00280836', '00223808, 1537534X',
       '00346861, 15390756', '20587546', '14710056, 14710064',
       '14741784, 14741776', '15206890, 00092665', '14741741, 14741733',
       '01492195, 1545861X', '10614036, 15461718', '00018392, 19303815',
       '19416520, 19416067', '00221082, 15406261', '15453278, 07320582',
       '15458636, 15460738', '03060012, 14604744', '17594782, 17594774',
       '10974180, 10747613', '01406736, 1474547X', '00346527, 1467937X',
       '15487091, 15487105', '17238617', '1553877X', '14764660, 14761122',
       '19358245, 19358237', '15221210, 00319333', '10959203, 00368075',
       '25201158', '17483387, 17483395', '00129682, 14680262', '00220515',
       '19457790, 19457782', '15356108, 18783686', '175

In [11]:
top_10_percent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1803 entries, 0 to 1802
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Rank                    1803 non-null   int64 
 1   Sourceid                1803 non-null   int64 
 2   Title                   1803 non-null   object
 3   Type                    1803 non-null   object
 4   Issn                    1803 non-null   object
 5   SJR                     1803 non-null   object
 6   SJR Best Quartile       1803 non-null   object
 7   H index                 1803 non-null   int64 
 8   Total Docs. (2022)      1803 non-null   int64 
 9   Total Docs. (3years)    1803 non-null   int64 
 10  Total Refs.             1803 non-null   int64 
 11  Total Cites (3years)    1803 non-null   int64 
 12  Citable Docs. (3years)  1803 non-null   int64 
 13  Cites / Doc. (2years)   1803 non-null   object
 14  Ref. / Doc.             1803 non-null   object
 15  Coun

In [12]:
top_10_percent[['Total Docs. (2022)', 'Total Docs. (3years)']]

,Total Docs. (2022),Total Docs. (3years)
0,44,118
1,36,122
2,121,328
3,420,1637
4,1410,4561
...,...,...
1798,56,119
1799,367,1245
1800,166,607
1801,0,45


In [13]:
top_10_percent[['Total Docs. (2022)', 'Total Docs. (3years)']].describe()

,Total Docs. (2022),Total Docs. (3years)
count,1803.000000,1803.000000
mean,305.364947,802.886855
std,585.720319,1464.200862
min,0.000000,3.000000
25%,66.000000,166.000000
50%,146.000000,379.000000
75%,311.500000,879.000000
max,8195.000000,20676.000000


In [14]:
top_10_percent[top_10_percent['Total Docs. (3years)']==20676]

,Rank,Sourceid,Title,Type,Issn,SJR,SJR Best Quartile,H index,Total Docs. (2022),Total Docs. (3years),...,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Region,Publisher,Coverage,Categories,Areas
1269,1270,25349,Science of the Total Environment,journal,"00489697, 18791026","1,946",Q1,317,8195,20676,...,225488,20446,"10,94","73,09",Netherlands,Western Europe,Elsevier,"1970, 1972-2023",Environmental Chemistry (Q1); Environmental En...,Environmental Science


# Bibliometric data on articles

## Webscraping by journal range

In [15]:
def webscraping_journal_range(website_path, start_journal, end_journal):
    """
    Scrapes scientific articles from Web of Science within a specified range of journals.

    Parameters:
    - website_path (str): The URL to the Web of Science search results page.
    - start_article (int): The starting journal number for scraping. Corresponds to index of journal in top_10_percent['Issn']
    - end_article (int): The ending journal number for scraping.

    Description:
    This function automates the web scraping process of scientific articles from Web of Science.
    It allows you to specify a range of journals you want to scrape (e.g., journal 1 to 100).
    The function will visit the provided website, reject cookies, and change the query to the journal 
    corresponding to the loop iteration, scrape articles within that query
    
    Note:
    - Ensure you have the ChromeDriver installed and its path specified in the function.
    - The function interacts with the Web of Science website, so it may need adjustments
      if the website's structure changes.
    - When you are scraping too many articles the function might stop because it asks to prove that
      you are human.

    Exceptions:
    To circumvent the CAPTCHA, if too many articles are scraped in one go, the following exceptions were put in place:
    - If the number of publications from query is above 30000, the function will return the index of that journal and reason "big_journal"
    - If the sum of the number of downloads (loop iterations) the function has gone through and the next journal exceeds 60, 
      the function will return the index of the next journal and reason "exceed_n_journals". 
    In case there is an error exporting the data:
    - Gives 30 more seconds to finish downloading the data from previous iteration, as elements might not be accessible yet
    - In case previous step doesn't work, the function returns the index of the journal and "STOP"

    Example Usage:
    website_path = "URL_TO_WEB_OF_SCIENCE_SEARCH_RESULTS"
    start_journal = 1
    end_journal = 100
    webscraping_wos_range(website_path, start_journal, end_journal)
    """

    # path for chrome driver
    path = Service(ChromeDriverManager().install())

    # create the driver
    driver = webdriver.Chrome(service = path)

    # open chromedriver window
    driver.get(website_path)
    time.sleep(10)

    driver.maximize_window()
    time.sleep(3)
    # reject cookies
    driver.find_element('xpath', '//*[@id="onetrust-reject-all-handler"]').click()
    
    journals = top_10_percent['Issn'][start_journal:end_journal].values

    journal_nr = start_journal
    loop_number = 0
    for issn in tqdm(journals):
        try: 
            # click on search bar
            driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-base-summary-component/app-search-friendly-display/div[1]/app-general-search-friendly-display/app-query-modifier/div/div[1]/div').click()
        except:
            time.sleep(30)
            driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-base-summary-component/app-search-friendly-display/div[1]/app-general-search-friendly-display/app-query-modifier/div/div[1]/div').click()

                
        # select correct year
        driver.find_element('xpath','//*[@id="advancedSearchInputArea"]').clear()
        time.sleep(0.5)
        issn_list = issn.replace(" ", "").split(',')
        comma_count = issn.count(',')
        if comma_count == 0:
            driver.find_element('xpath','//*[@id="advancedSearchInputArea"]').send_keys(f"IS=({issn_list[0]})")
        elif comma_count == 1:
            driver.find_element('xpath','//*[@id="advancedSearchInputArea"]').send_keys(f"IS=({issn_list[0]}) OR IS=({issn_list[1]})")
        else:
            driver.find_element('xpath','//*[@id="advancedSearchInputArea"]').send_keys(f"IS=({issn_list[0]}) OR IS=({issn_list[1]}) OR IS=({issn_list[2]})")

        # click search 
        driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-base-summary-component/app-search-friendly-display/div[1]/app-general-search-friendly-display/app-query-modifier/div/div[1]/div/div[4]/div[2]/app-advanced-search-form/form/div[2]/div[1]/div/div/button[2]/span[1]').click()
        time.sleep(3)

        # calculate total number of publications, nem é bem preciso para nada
        n_publications = int(driver.find_element(By.CLASS_NAME, "brand-blue").text.replace(',', ''))

        if n_publications > 30_000:
            print(f'This journal ({journal_nr}) has too many articles. Export individually.')
            return journal_nr, 'big_journal'
        
        nr_iterations = math.ceil((n_publications - 0) / 500)

        if nr_iterations + loop_number >60:
            print(f'The next journal ({journal_nr}) will exceed the limit of automatic exports')
            return journal_nr, 'exceed_n_journals'


        for page in range(nr_iterations):
            loop_number += 1

            time.sleep(2)
            driver.implicitly_wait(100)
            # calculate the range of articles to scrape on the current page
            page_start_article = page * 500 + 1 + 0
            page_end_article = min((page + 1) * 500 + 0, n_publications)
            
            try:
                # open "Export" options
                driver.find_element('xpath', '//*[@id="snRecListTop"]/app-export-menu/div/button').click()

                time.sleep(2)

                print(f"Scraping articles {page_start_article} to {page_end_article}")
                
                # export to tab-delimited file
                driver.find_element('xpath', '//*[@id="exportToTabWinButton"]').click()

                # export to bibtex file
                #driver.find_element('xpath', '//*[@id="exportToBibtexButton"]').click()

            except:
                try: 
                    time.sleep(30)
                    # open "Export" options
                    driver.find_element('xpath', '//*[@id="snRecListTop"]/app-export-menu/div/button').click()

                    time.sleep(2)

                    print(f"Scraping articles {page_start_article} to {page_end_article}")
                    
                    # export to tab-delimited file
                    driver.find_element('xpath', '//*[@id="exportToTabWinButton"]').click()
                    
                except:
                    print(f'Page number {page_start_article} until {page_end_article} was not exported')
                    return journal_nr, 'STOP'
                   
            
            time.sleep(3)
            driver.find_element('xpath', '//*[@id="radio3"]/label/span[1]/span[2]').click()
            # choose range of papers to export
            # if page == 0:
            #     driver.find_element('xpath', '//*[@id="radio3"]/label/span[1]/span[2]').click()
            # else:
            #     driver.find_element('xpath', '//*[@id="radio3"]/label/span[1]/span[1]').click()

                #//*[@id="radio3"]/label/span[1]/span[2]
            #driver.find_element(By.CLASS_NAME, "mat-radio-inner-circle")
        
            time.sleep(1)
            
            # starting range
            driver.find_element(By.NAME, "markFrom").clear()
            time.sleep(0.5)
            driver.find_element(By.NAME, "markFrom").send_keys(page_start_article)
            time.sleep(0.5)
            
            # ending range
            driver.find_element(By.NAME, "markTo").clear()
            time.sleep(0.5)
            driver.find_element(By.NAME, "markTo").send_keys(page_end_article)
            time.sleep(0.5)
            
            # dropdown "Record Content:"
            driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[1]/wos-select/button').click()

            time.sleep(2)
            
            # choose the record content to be "Full Record and Cited References"
            driver.find_element('xpath', '//*[@id="global-select"]/div/div/div[4]/span').click()

            time.sleep(2)
            
            # click export
            driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[2]/button[1]').click()

            time.sleep(13)

        journal_nr += 1
        
    # close the chrome window
    driver.quit()
    return journal_nr+10, 'STOP'

In [22]:
top_10_percent['Issn'][1407]
#1931-3128

'00048674, 14401614'

In [17]:
top_10_percent[top_10_percent['Issn'].astype(str).str.contains('00377732')].index

Index([1055], dtype='int64')

In [18]:
website = "https://www.webofscience.com/wos/woscc/summary/75146b30-3227-443c-bb5a-771738929e0b-b2272def/relevance/1"

In [19]:
#webscraping_journal_range(website,116, 200)

In [20]:
too_big = []

In [23]:
starting_journal = 1465
ending_journal = 1500
while starting_journal<ending_journal:
    journal_nr, reason = webscraping_journal_range(website,starting_journal, ending_journal)
    time.sleep(100)
    if reason == 'big_journal':
        too_big.append(journal_nr)
        starting_journal = journal_nr + 1
    elif reason == 'exceed_n_journals':
        starting_journal = journal_nr
    elif reason == 'STOP':
        print(journal_nr)
        starting_journal=journal_nr
        break

  0%|          | 0/93 [00:00<?, ?it/s]

Scraping articles 1 to 500
Scraping articles 501 to 1000
Scraping articles 1001 to 1500
Scraping articles 1501 to 2000
Scraping articles 2001 to 2500
Scraping articles 2501 to 3000
Scraping articles 3001 to 3500
Scraping articles 3501 to 4000
Scraping articles 4001 to 4500
Scraping articles 4501 to 5000
Scraping articles 5001 to 5500
Scraping articles 5501 to 6000
Scraping articles 6001 to 6500
Scraping articles 6501 to 7000
Scraping articles 7001 to 7500
Scraping articles 7501 to 8000
Scraping articles 8001 to 8500
Scraping articles 8501 to 9000
Scraping articles 9001 to 9500
Scraping articles 9501 to 10000
Scraping articles 10001 to 10500
Scraping articles 10501 to 11000
Scraping articles 11001 to 11500
Scraping articles 11501 to 12000
Scraping articles 12001 to 12500
Scraping articles 12501 to 13000
Scraping articles 13001 to 13500
Scraping articles 13501 to 14000
Scraping articles 14001 to 14500
Scraping articles 14501 to 15000
Scraping articles 15001 to 15372


  1%|          | 1/93 [14:19<21:58:24, 859.83s/it]

Scraping articles 1 to 500
Scraping articles 501 to 1000
Scraping articles 1001 to 1500
Scraping articles 1501 to 2000
Scraping articles 2001 to 2500
Scraping articles 2501 to 2571


  2%|▏         | 2/93 [17:15<11:33:16, 457.10s/it]

...

Scraping articles 1 to 500
Scraping articles 501 to 1000
Scraping articles 1001 to 1500
Scraping articles 1501 to 2000
Scraping articles 2001 to 2500
Scraping articles 2501 to 3000
Scraping articles 3001 to 3500
Scraping articles 3501 to 4000
Scraping articles 4001 to 4500
Scraping articles 4501 to 5000
Scraping articles 5001 to 5500
Scraping articles 5501 to 6000
Scraping articles 6001 to 6500
Scraping articles 6501 to 7000
Scraping articles 7001 to 7500
Scraping articles 7501 to 8000
Scraping articles 8001 to 8500
Scraping articles 8501 to 9000
Scraping articles 9001 to 9500
Scraping articles 9501 to 10000
Scraping articles 10001 to 10500
Scraping articles 10501 to 11000
Scraping articles 11001 to 11500
Scraping articles 11501 to 12000
Scraping articles 12001 to 12500
Scraping articles 12501 to 13000
Scraping articles 13001 to 13331


  3%|▎         | 1/37 [12:31<7:30:57, 751.61s/it]

Scraping articles 1 to 420


  5%|▌         | 2/37 [13:07<3:49:37, 393.65s/it]

The next journal (1465) will exceed the limit of automatic exports


In [22]:
too_big

[1367, 1386, 1394]

In [80]:
too_big

[1203, 1211, 1226, 1239]

In [26]:
starting_journal

675

## Check Big Journals

In [ ]:
dimensions_df = pd.DataFrame()
journal_dimensions = []
for journal in top_10_percent['Issn']: 
    dimension = webscraping_journal_dimension(journal)
    journal_dimensions.append(dimension)

## Journals individual export

This section is dedicated to the download of data from journals that have too many articles to be extracted in one go. 

In [18]:
def webscraping_article_range(website_path, journal_number, start_article, end_article):
    """
    Scrapes scientific articles from Web of Science within a specified range.

    Parameters:
    - website_path (str): The URL to the Web of Science search results page.
    - start_article (int): The starting article number for scraping.
    - end_article (int): The ending article number for scraping.

    Description:
    This function automates the web scraping process of scientific articles from Web of Science.
    It allows you to specify a range of articles you want to scrape (e.g., articles 5000 to 6000).
    The function will visit the provided website, reject cookies, and then scrape articles within
    the specified range.

    Note:
    - Ensure you have the ChromeDriver installed and its path specified in the function.
    - The function interacts with the Web of Science website, so it may need adjustments
      if the website's structure changes.
    - When you are scraping too many articles the function might stop because it asks to prove that
      you are human.

    Example Usage:
    website_path = "URL_TO_WEB_OF_SCIENCE_SEARCH_RESULTS"
    start_article = 5000
    end_article = 6000
    webscraping_wos_range(website_path, start_article, end_article)
    """

    # path for chrome driver
    path = Service(ChromeDriverManager().install())

    # create the driver
    driver = webdriver.Chrome(service = path)

    # open chromedriver window
    driver.get(website_path)
    time.sleep(10)
    
    # reject cookies
    driver.find_element('xpath', '//*[@id="onetrust-reject-all-handler"]').click()
    
    issn = top_10_percent['Issn'][journal_number]

    driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-base-summary-component/app-search-friendly-display/div[1]/app-general-search-friendly-display/app-query-modifier/div/div[1]/div').click()

    # select correct year
    driver.find_element('xpath','//*[@id="advancedSearchInputArea"]').clear()
    time.sleep(0.5)
    issn_list = issn.replace(" ", "").split(',')
    comma_count = issn.count(',')
    if comma_count == 0:
        driver.find_element('xpath','//*[@id="advancedSearchInputArea"]').send_keys(f"IS=({issn_list[0]})")
    elif comma_count == 1:
        driver.find_element('xpath','//*[@id="advancedSearchInputArea"]').send_keys(f"IS=({issn_list[0]}) OR IS=({issn_list[1]})")
    else:
        driver.find_element('xpath','//*[@id="advancedSearchInputArea"]').send_keys(f"IS=({issn_list[0]}) OR IS=({issn_list[1]}) OR IS=({issn_list[2]})")

    # click search 
    driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-base-summary-component/app-search-friendly-display/div[1]/app-general-search-friendly-display/app-query-modifier/div/div[1]/div/div[4]/div[2]/app-advanced-search-form/form/div[2]/div[1]/div/div/button[2]/span[1]').click()
    time.sleep(3)

    # calculate total number of publications, nem é bem preciso para nada
    n_publications = int(driver.find_element(By.CLASS_NAME, "brand-blue").text.replace(',', ''))
    
    nr_iterations = math.ceil((end_article - start_article) / 500)
    
    for page in range(nr_iterations):

        time.sleep(2)
        
        # calculate the range of articles to scrape on the current page
        page_start_article = page * 500 + 1 + start_article
        page_end_article = min((page + 1) * 500 + start_article, end_article)
        
        try:
            driver.implicitly_wait(100)
            # open "Export" options
            driver.find_element('xpath', '//*[@id="snRecListTop"]/app-export-menu/div/button').click()

            time.sleep(2)

            print(f"Scraping articles {page_start_article} to {page_end_article}")
            
            driver.implicitly_wait(100)
            # export to tab-delimited file
            driver.find_element('xpath', '//*[@id="exportToTabWinButton"]').click()

            # export to bibtex file
            #driver.find_element('xpath', '//*[@id="exportToBibtexButton"]').click()

        except:
            print(f'Page number {page_start_article} until {page_end_article} was not exported')
            continue
        
        time.sleep(3)
        driver.find_element('xpath', '//*[@id="radio3"]/label/span[1]/span[2]').click()
    
        time.sleep(1)
        
        # starting range
        driver.find_element(By.NAME, "markFrom").clear()
        time.sleep(0.5)
        driver.find_element(By.NAME, "markFrom").send_keys(page_start_article)
        time.sleep(0.5)
        
        # ending range
        driver.find_element(By.NAME, "markTo").clear()
        time.sleep(0.5)
        driver.find_element(By.NAME, "markTo").send_keys(page_end_article)
        time.sleep(0.5)
        
        # dropdown "Record Content:"
        driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[1]/wos-select/button').click()

        time.sleep(2)
        
        # choose the record content to be "Full Record and Cited References"
        driver.find_element('xpath', '//*[@id="global-select"]/div/div/div[4]/span').click()

        time.sleep(2)
        
        # click export
        driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[2]/button[1]').click()

        time.sleep(30)

        
    # close the chrome window
    driver.quit()

In [19]:
website = "https://www.webofscience.com/wos/woscc/summary/75146b30-3227-443c-bb5a-771738929e0b-b2272def/relevance/1"

In [20]:
webscraping_article_range(website, 17, 0, 7931)

Scraping articles 1 to 500
Scraping articles 501 to 1000
Scraping articles 1001 to 1500
Scraping articles 1501 to 2000
Page number 2001 until 2500 was not exported
Scraping articles 2501 to 3000
Scraping articles 3001 to 3500
Scraping articles 3501 to 4000
Scraping articles 4001 to 4500
Scraping articles 4501 to 5000
Scraping articles 5001 to 5500
Scraping articles 5501 to 6000
Scraping articles 6001 to 6500
Scraping articles 6501 to 7000
Scraping articles 7001 to 7500
Scraping articles 7501 to 7931


In [20]:
webscraping_article_range(website, 4, 34000, 60000)

Scraping articles 34001 to 34500
Scraping articles 34501 to 35000
Scraping articles 35001 to 35500
Scraping articles 35501 to 36000
Scraping articles 36001 to 36500
Scraping articles 36501 to 37000
Scraping articles 37001 to 37500
Scraping articles 37501 to 38000
Scraping articles 38001 to 38500
Scraping articles 38501 to 39000
Scraping articles 39001 to 39500
Scraping articles 39501 to 40000
Scraping articles 40001 to 40500
Scraping articles 40501 to 41000
Scraping articles 41001 to 41500
Scraping articles 41501 to 42000
Scraping articles 42001 to 42500
Scraping articles 42501 to 43000
Scraping articles 43001 to 43500
Scraping articles 43501 to 44000
Scraping articles 44001 to 44500
Scraping articles 44501 to 45000
Scraping articles 45001 to 45500
Scraping articles 45501 to 46000
Scraping articles 46001 to 46500
Scraping articles 46501 to 47000
Scraping articles 47001 to 47500
Scraping articles 47501 to 48000
Scraping articles 48001 to 48500
Scraping articles 48501 to 49000
Scraping a

In [21]:
webscraping_article_range(website, 4, 60000, 90000)

Scraping articles 60001 to 60500
Scraping articles 60501 to 61000
Scraping articles 61001 to 61500
Scraping articles 61501 to 62000
Scraping articles 62001 to 62500
Scraping articles 62501 to 63000
Scraping articles 63001 to 63500
Scraping articles 63501 to 64000
Scraping articles 64001 to 64500
Scraping articles 64501 to 65000
Scraping articles 65001 to 65500
Scraping articles 65501 to 66000
Scraping articles 66001 to 66500
Scraping articles 66501 to 67000
Scraping articles 67001 to 67500
Scraping articles 67501 to 68000
Scraping articles 68001 to 68500
Scraping articles 68501 to 69000
Scraping articles 69001 to 69500
Scraping articles 69501 to 70000
Scraping articles 70001 to 70500
Scraping articles 70501 to 71000
Scraping articles 71001 to 71500
Scraping articles 71501 to 72000
Scraping articles 72001 to 72500
Scraping articles 72501 to 73000
Scraping articles 73001 to 73500
Scraping articles 73501 to 74000
Scraping articles 74001 to 74500
Scraping articles 74501 to 75000
Scraping a

In [24]:
webscraping_article_range(website, 4, 33501, 34000)

Scraping articles 33502 to 34000


In [ ]:
webscraping_article_range(website, 4, 111000, 114923)

In [16]:
webscraping_wos_range(website, 1990, 1999)

Scraping articles 1 to 29
Scraping articles 1 to 33
Scraping articles 1 to 24
Scraping articles 1 to 34
Scraping articles 1 to 43
Scraping articles 1 to 56
Scraping articles 1 to 54
Scraping articles 1 to 64
Scraping articles 1 to 84
Scraping articles 1 to 90


In [17]:
webscraping_wos_range(website, 1980, 1989)

Scraping articles 1 to 16
Scraping articles 1 to 6
Scraping articles 1 to 10
Scraping articles 1 to 7
Scraping articles 1 to 3
Scraping articles 1 to 6
Scraping articles 1 to 12
Scraping articles 1 to 15
Scraping articles 1 to 14
Scraping articles 1 to 11


## Join data into less datasets

In [77]:
folder_list =  [f"{i}-{i+99}" for i in range(100, 1701, 100)]
problem_files = []

In [ ]:
for range_journals in tqdm(folder_list):
    filnames = os.listdir(f"C:/Users/isabe/Desktop/Tese/journals{range_journals}")
    journals_data = pd.DataFrame()
    for file in filnames:
        try: 
            data = pd.read_csv(f"C:/Users/isabe/Desktop/Tese/journals{range_journals}/{file}", sep = '\t')
            journals_data = pd.concat([journals_data, data], ignore_index= True)
        except:
            problem_files.append(file)
    journals_data.to_csv(f'./retractions_data/journals_data{range_journals}.csv', index= False)

In [ ]:
folder = "400-499"
filnames = os.listdir(f"C:/Users/isabe/Desktop/Tese/journals{folder}")
journals_data = pd.DataFrame()
for file in filnames:
    try: 
        data = pd.read_csv(f"C:/Users/isabe/Desktop/Tese/journals{folder}/{file}", sep = '\t')
        journals_data = pd.concat([journals_data, data], ignore_index= True)
    except:
        problem_files.append(file)

In [ ]:
journals_data.to_csv(f'./retractions_data/journals_data{folder}.csv', index= False)

In [ ]:
problem_files

## Check if Journals were all scraped

_____end here_____

In [18]:
downloads_folder =  os.path.expanduser('~') + '/Downloads' #"C:/Users/isabe/Downloads"
count = 0

# List all files in the Downloads folder
for filename in os.listdir(downloads_folder):
    # Check if the filename contains "savedrecs" and ends with ".csv"
    if "savedrecs" in filename and filename.endswith(".txt"):
        count += 1

print(f"Number of 'savedrecs' CSV files in Downloads: {count}")

Number of 'savedrecs' CSV files in Downloads: 61


In [27]:
retractions_data = pd.DataFrame()
for file in range(count):
    if file == 0:
        data = pd.read_csv("C:/Users/isabe/Downloads/savedrecs.txt", sep = '\t')
        retractions_data = pd.concat([retractions_data, data], ignore_index= True)
    else:
        data = pd.read_csv(f"C:/Users/isabe/Downloads/savedrecs ({file}).txt", sep = '\t')
        retractions_data = pd.concat([retractions_data, data], ignore_index= True)

In [31]:
rename_columns = {
    "FN": "File Name",
    "VR": "Version Number",
    "PT": "Publication Type", # (J=Journal; B=Book; S=Series; P=Patent)
    "AU": "Authors",
    "AF": "Author Full Name",
    "BA": "Book Authors",
    "BF": "Book Authors Full Name",
    "CA": "Group Authors",
    "GP": "Book Group Authors",
    "BE": "Editors",
    "TI": "Document Title",
    "SO": "Publication Name",
    "SE": "Book Series Title",
    "BS": "Book Series Subtitle",
    "LA": "Language",
    "DT": "Document Type",
    "CT": "Conference Title",
    "CY": "Conference Date",
    "CL": "Conference Location",
    "SP": "Conference Sponsors",
    "HO": "Conference Host",
    "DE": "Author Keywords",
    "ID": "Keywords Plus",
    "AB": "Abstract",
    "C1": "Author Address",
    "RP": "Reprint Address",
    "EM": "E-mail Address",
    "RI": "ResearcherID Number",
    "OI": "ORCID Identifier (Open Researcher and Contributor ID)",
    "FU": "Funding Agency and Grant Number",
    'FP': 'Funding Name Preferred',
    "FX": "Funding Text",
    "CR": "Cited References",
    "NR": "Cited Reference Count",
    "TC": "Web of Science Core Collection Times Cited Count",
    "Z9": "Total Times Cited Count",
    "U1": "Usage Count (Last 180 Days)",
    "U2": "Usage Count (Since 2013)",
    "PU": "Publisher",
    "PI": "Publisher City",
    "PA": "Publisher Address",
    "SN": "International Standard Serial Number (ISSN)",
    "EI": "Electronic International Standard Serial Number (eISSN)",
    "BN": "International Standard Book Number (ISBN)",
    "J9": "29-Character Source Abbreviation",
    "JI": "ISO Source Abbreviation",
    "PD": "Publication Date",
    "PY": "Year Published",
    "VL": "Volume",
    "IS": "Issue",
    "SI": "Special Issue",
    "PN": "Part Number",
    "SU": "Supplement",
    "MA": "Meeting Abstract",
    "BP": "Beginning Page",
    "EP": "Ending Page",
    "AR": "Article Number",
    "DI": "Digital Object Identifier (DOI)",
    'DL': 'DOI Link',
    "D2": "Book Digital Object Identifier (DOI)",
    "EA": "Early access date",
    "EY": "Early access year",
    "PG": "Page Count",
    "P2": "Chapter Count (Book Citation Index)",
    "WC": "Web of Science Categories",
    'WE': 'Web of Science Index',
    "SC": "Research Areas",
    "GA": "Document Delivery Number",
    "PM": "PubMed ID",
    "UT": "Accession Number",
    "OA": "Open Access Indicator",
    "HP": "ESI Hot Paper", # Note that this field is valued only for ESI subscribers.
    "HC": "ESI Highly Cited Paper", # Note that this field is valued only for ESI subscribers.
    "DA": "Date this report was generated",
    "ER": "End of Record",
    "EF": "End of File"
}

retractions_data.rename(columns = rename_columns, inplace = True)
retractions_data.columns

Index(['Publication Type', 'Authors', 'Book Authors', 'Editors',
       'Book Group Authors', 'Author Full Name', 'Book Authors Full Name',
       'Group Authors', 'Document Title', 'Publication Name',
       'Book Series Title', 'Book Series Subtitle', 'Language',
       'Document Type', 'Conference Title', 'Conference Date',
       'Conference Location', 'Conference Sponsors', 'Conference Host',
       'Author Keywords', 'Keywords Plus', 'Abstract', 'Author Address', 'C3',
       'Reprint Address', 'E-mail Address', 'ResearcherID Number',
       'ORCID Identifier (Open Researcher and Contributor ID)',
       'Funding Agency and Grant Number', 'FP', 'Funding Text',
       'Cited References', 'Cited Reference Count',
       'Web of Science Core Collection Times Cited Count',
       'Total Times Cited Count', 'Usage Count (Last 180 Days)',
       'Usage Count (Since 2013)', 'Publisher', 'Publisher City',
       'Publisher Address', 'International Standard Serial Number (ISSN)',
       '

In [32]:
retractions_data

,Publication Type,Authors,Book Authors,Editors,Book Group Authors,Author Full Name,Book Authors Full Name,Group Authors,Document Title,Publication Name,...,Web of Science Categories,WE,Research Areas,Document Delivery Number,PubMed ID,Open Access Indicator,ESI Highly Cited Paper,ESI Hot Paper,Date this report was generated,Accession Number
0,J,"Qiang, RL; Guang, LF; Hua, SY; Maia, D",NaN,NaN,NaN,"Qiang, Ran Li; Guang, Li Fa; Hua, Sun Yan; Mai...",NaN,NaN,RETRACTED: Art Research Based on Machine Learn...,SECURITY AND COMMUNICATION NETWORKS,...,"Computer Science, Information Systems; Telecom...",Science Citation Index Expanded (SCI-EXPANDED),Computer Science; Telecommunications,1M9VB,NaN,gold,NaN,NaN,2023-11-05,WOS:000800311400007
1,J,"Fan, L; Xia, MY; Huang, P; Hu, JM",NaN,NaN,NaN,"Fan, Ling; Xia, Meiyi; Huang, Ping; Hu, Jianmin",NaN,NaN,RETRACTED: Research on Educational Information...,SECURITY AND COMMUNICATION NETWORKS,...,"Computer Science, Information Systems; Telecom...",Science Citation Index Expanded (SCI-EXPANDED),Computer Science; Telecommunications,UW7ON,NaN,gold,NaN,NaN,2023-11-05,WOS:000700342400003
2,J,"Peng, AH; Liu, CW; Yang, L",NaN,NaN,NaN,"Peng, Anhua; Liu, Chengwen; Yang, Le",NaN,NaN,RETRACTED: Intelligent electrical equipment fi...,EURASIP JOURNAL ON ADVANCES IN SIGNAL PROCESSING,...,"Engineering, Electrical & Electronic",Science Citation Index Expanded (SCI-EXPANDED),Engineering,UU7XW,NaN,gold,NaN,NaN,2023-11-05,WOS:000699010400001
3,J,"Cui, LH",NaN,NaN,NaN,"Cui, Lianhe",NaN,NaN,RETRACTED: Research on English translation dis...,EURASIP JOURNAL ON IMAGE AND VIDEO PROCESSING,...,"Engineering, Electrical & Electronic; Imaging ...",Science Citation Index Expanded (SCI-EXPANDED),Engineering; Imaging Science & Photographic Te...,HJ4MN,NaN,gold,NaN,NaN,2023-11-05,WOS:000457148200002
4,J,"Zhu, SS",NaN,NaN,NaN,"Zhu, Shanshan",NaN,NaN,RETRACTED: Real-time detection of aerobics pos...,EURASIP JOURNAL ON ADVANCES IN SIGNAL PROCESSING,...,"Engineering, Electrical & Electronic",Science Citation Index Expanded (SCI-EXPANDED),Engineering,XA5VI,NaN,gold,NaN,NaN,2023-11-05,WOS:000720713700001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23553,J,"GROSSARTHMATICEK, R; EYSENCK, HJ",NaN,NaN,NaN,"GROSSARTHMATICEK, R; EYSENCK, HJ",NaN,NaN,RETRACTED: LENGTH OF SURVIVAL AND LYMPHOCYTE P...,PSYCHOLOGICAL REPORTS,...,"Psychology, Multidisciplinary",Social Science Citation Index (SSCI),Psychology,AN942,2780939.0,NaN,NaN,NaN,2023-11-05,WOS:A1989AN94200055
23554,J,"CHANDRA, RK; PURI, S; HAMED, A",NaN,NaN,NaN,"CHANDRA, RK; PURI, S; HAMED, A",NaN,NaN,RETRACTED: INFLUENCE OF MATERNAL DIET DURING L...,BMJ-BRITISH MEDICAL JOURNAL,...,"Medicine, General & Internal",Science Citation Index Expanded (SCI-EXPANDED),General & Internal Medicine,AG617,2504375.0,"Green Published, hybrid",NaN,NaN,2023-11-05,WOS:A1989AG61700016
23555,J,"AITKEN, JC; THOMPSON, J",NaN,NaN,NaN,"AITKEN, JC; THOMPSON, J",NaN,NaN,RETRACTED: THE EFFECTS OF DIETARY MANIPULATION...,EUROPEAN JOURNAL OF APPLIED PHYSIOLOGY,...,Physiology; Sport Sciences,Science Citation Index Expanded (SCI-EXPANDED),Physiology; Sport Sciences,U8775,2737192.0,NaN,NaN,NaN,2023-11-05,WOS:A1989U877500008
23556,J,"BOLDT, J; KLING, D; VONBORMANN, B; ZUGE, M; SC...",NaN,NaN,NaN,"BOLDT, J; KLING, D; VONBORMANN, B; ZUGE, M; SC...",NaN,NaN,RETRACTED: BLOOD CONSERVATION IN CARDIAC OPERA...,JOURNAL OF THORACIC AND CARDIOVASCULAR SURGERY,...,Cardiac & Cardiovascular Systems; Respiratory ...,Science Citation Index Expanded (SCI-EXPANDED),Cardiovascular System & Cardiology; Respirator...,AA700,2786116.0,NaN,NaN,NaN,2023-11-05,WOS:A1989AA70000004


In [33]:
retractions_data.to_csv('./retractions_data/wos_retractions_data.csv', index= False)

### Importing the information to python

In [160]:
# def import_text_files_as_dataframe(folder_path, prefix = ''):
#     """
#     Import all text files from a folder whose names start with a specific prefix
#     and concatenate them into a single pandas DataFrame.

#     Parameters:
#     - folder_path (str): The path to the folder containing the text files.
#     - prefix (str, optional): The prefix that the filenames should start with. Default is an empty string.

#     Returns:
#     - pandas.DataFrame: A DataFrame containing the content of all matching text files.

#     Description:
#     This function reads all text files within a specified folder that have filenames
#     starting with the provided prefix. It imports the content of these files and
#     concatenates them into a single pandas DataFrame.

#     - folder_path: The path to the folder containing the text files.
#     - prefix: The prefix that the filenames should start with. If not provided, all text files
#       in the folder will be imported.

#     Example Usage:
#     folder_path = '/path/to/your/folder'
#     prefix = 'your_prefix'
#     df = import_text_files_as_dataframe(folder_path, prefix)
#     """

#     files_in_folder = os.listdir(folder_path)
#     matching_files = [file for file in files_in_folder if file.startswith(prefix) and file.endswith('.txt')]

#     if not matching_files:
#         print("No matching files found.")
#         return None

#     concat_df = pd.DataFrame()
#     for file_name in matching_files:
#         file_path = os.path.join(folder_path, file_name)
#         df = pd.read_csv(file_path, sep = '\t')
#         concat_df = pd.concat([concat_df, df])
        
#     return concat_df

In [161]:
# folder_path = os.path.join(work_path, "WoS Electric Car tsv")
# prefix = 'savedrecs'

# df = import_text_files_as_dataframe(folder_path, prefix)
# df.head()

In [162]:
# rename_columns = {
#     "FN": "File Name",
#     "VR": "Version Number",
#     "PT": "Publication Type", # (J=Journal; B=Book; S=Series; P=Patent)
#     "AU": "Authors",
#     "AF": "Author Full Name",
#     "BA": "Book Authors",
#     "BF": "Book Authors Full Name",
#     "CA": "Group Authors",
#     "GP": "Book Group Authors",
#     "BE": "Editors",
#     "TI": "Document Title",
#     "SO": "Publication Name",
#     "SE": "Book Series Title",
#     "BS": "Book Series Subtitle",
#     "LA": "Language",
#     "DT": "Document Type",
#     "CT": "Conference Title",
#     "CY": "Conference Date",
#     "CL": "Conference Location",
#     "SP": "Conference Sponsors",
#     "HO": "Conference Host",
#     "DE": "Author Keywords",
#     "ID": "Keywords Plus",
#     "AB": "Abstract",
#     "C1": "Author Address",
#     "RP": "Reprint Address",
#     "EM": "E-mail Address",
#     "RI": "ResearcherID Number",
#     "OI": "ORCID Identifier (Open Researcher and Contributor ID)",
#     "FU": "Funding Agency and Grant Number",
#     "FX": "Funding Text",
#     "CR": "Cited References",
#     "NR": "Cited Reference Count",
#     "TC": "Web of Science Core Collection Times Cited Count",
#     "Z9": "Total Times Cited Count",
#     "U1": "Usage Count (Last 180 Days)",
#     "U2": "Usage Count (Since 2013)",
#     "PU": "Publisher",
#     "PI": "Publisher City",
#     "PA": "Publisher Address",
#     "SN": "International Standard Serial Number (ISSN)",
#     "EI": "Electronic International Standard Serial Number (eISSN)",
#     "BN": "International Standard Book Number (ISBN)",
#     "J9": "29-Character Source Abbreviation",
#     "JI": "ISO Source Abbreviation",
#     "PD": "Publication Date",
#     "PY": "Year Published",
#     "VL": "Volume",
#     "IS": "Issue",
#     "SI": "Special Issue",
#     "PN": "Part Number",
#     "SU": "Supplement",
#     "MA": "Meeting Abstract",
#     "BP": "Beginning Page",
#     "EP": "Ending Page",
#     "AR": "Article Number",
#     "DI": "Digital Object Identifier (DOI)",
#     "D2": "Book Digital Object Identifier (DOI)",
#     "EA": "Early access date",
#     "EY": "Early access year",
#     "PG": "Page Count",
#     "P2": "Chapter Count (Book Citation Index)",
#     "WC": "Web of Science Categories",
#     "SC": "Research Areas",
#     "GA": "Document Delivery Number",
#     "PM": "PubMed ID",
#     "UT": "Accession Number",
#     "OA": "Open Access Indicator",
#     "HP": "ESI Hot Paper", # Note that this field is valued only for ESI subscribers.
#     "HC": "ESI Highly Cited Paper", # Note that this field is valued only for ESI subscribers.
#     "DA": "Date this report was generated",
#     "ER": "End of Record",
#     "EF": "End of File"
# }

# df.rename(columns = rename_columns, inplace = True)
# df.columns

In [163]:
# df.shape

In [164]:
# df.drop_duplicates(subset = "Accession Number", inplace = True)
# df.shape

In [165]:
# # saving to csv the dataframe to later use

# csv_path = os.path.join(work_path, "WoSElectricCarData.csv")
# df.to_csv(csv_path, index = False)

## Analysis

In [97]:
# # importing the data from the csv file

# csv_path = os.path.join(work_path, "WoSElectricCarData.csv")

# df = pd.read_csv(csv_path)
# df.head()

In [98]:
# df.info()

In [99]:
# df['Year Published'].describe()

In [100]:
# df_year = df.groupby("Year Published", as_index = False).count()[["Year Published", "Publication Name"]]
# df_year

In [101]:
# # fig = px.line(df_year, x = 'Year Published', y = 'Publication Name', markers = True, text = 'Publication Name', title = "Number of Articles by Year", labels = {"Publication Name": "", "Year Published": "Year"}, width = 800, height = 600)
# # fig.update_traces(textposition = 'top center')

# fig = px.line(df_year, x = 'Year Published', y = 'Publication Name', markers = True, title = "Number of Articles by Year", labels = {"Publication Name": "", "Year Published": "Year"}, width = 800, height = 600)

# number_articles_year_path = os.path.join(work_path, "NumberArticlesYear.pdf")
# fig.write_image(number_articles_year_path)

# fig.show()

In [102]:
# # importing the data from the csv file

# csv_path = os.path.join(work_path, "WoS_Rdata.csv")

# df = pd.read_csv(csv_path)
# df.head()

In [103]:
# df.columns

In [104]:
# df['PY'].describe()

In [105]:
# df_year = df.groupby("PY", as_index = False).count()[["PY", "TI"]]
# df_year

In [106]:
# # fig = px.line(df_year, x = 'Year Published', y = 'Publication Name', markers = True, text = 'Publication Name', title = "Number of Articles by Year", labels = {"Publication Name": "", "Year Published": "Year"}, width = 800, height = 600)
# # fig.update_traces(textposition = 'top center')

# fig = px.line(df_year, x = 'PY', y = 'TI', markers = True, title = "Number of Articles by Year", labels = {"TI": "", "PY": "Year"}, width = 800, height = 600)

# number_articles_year_path = os.path.join(work_path, "NumberArticlesYear.pdf")
# fig.write_image(number_articles_year_path)

# fig.show()